In [ ]:
import nannyml as nml
from IPython.display import display

reference_df = nml.load_synthetic_car_loan_dataset()[0]
analysis_df = nml.load_synthetic_car_loan_dataset()[1]

display(reference_df.head(3))

/var/home/nannyml/.conda/envs/poetry/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1


In [ ]:
print(reference_df.head(3).to_markdown(tablefmt="grid"))

+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+--------------

In [ ]:
estimator = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='repaid',
    timestamp_column_name='timestamp',
    metrics=['roc_auc', 'confusion_matrix',],
    chunk_size=5000,
    problem_type='classification_binary',
    normalize_confusion_matrix='pred',
)

estimator.fit(reference_df)

In [ ]:
results = estimator.estimate(analysis_df)
display(results.to_df())

chunk                                                        \
              key chunk_index start_index end_index          start_date   
0        [0:4999]           0           0      4999 2018-01-01 00:00:00   
1     [5000:9999]           1        5000      9999 2018-01-31 06:36:00   
2   [10000:14999]           2       10000     14999 2018-03-02 13:12:00   
3   [15000:19999]           3       15000     19999 2018-04-01 19:48:00   
4   [20000:24999]           4       20000     24999 2018-05-02 02:24:00   
5   [25000:29999]           5       25000     29999 2018-06-01 09:00:00   
6   [30000:34999]           6       30000     34999 2018-07-01 15:36:00   
7   [35000:39999]           7       35000     39999 2018-07-31 22:12:00   
8   [40000:44999]           8       40000     44999 2018-08-31 04:48:00   
9   [45000:49999]           9       45000     49999 2018-09-30 11:24:00   
10       [0:4999]           0           0      4999 2018-10-30 18:00:00   
11    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
12  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
13  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
14  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
15  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
16  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
17  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
18  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
19  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                        roc_auc                           ...  \
                  end_date     period     value sampling_error  realized  ...   
0  2018-01-31 06:27:16.848  reference  0.969051       0.001811  0.976253  ...   
1  2018-03-02 13:03:16.848  reference  0.968909       0.001811  0.969045  ...   
2  2018-04-01 19:39:16.848  reference  0.968657       0.001811  0.971742  ...   
3  2018-05-02 02:15:16.848  reference  0.968973       0.001811  0.971642  ...   
4  2018-06-01 08:51:16.848  reference  0.968962       0.001811  0.969085  ...   
5  2018-07-01 15:27:16.848  reference  0.969079       0.001811  0.967364  ...   
6  2018-07-31 22:03:16.848  reference  0.968725       0.001811  0.968692  ...   
7  2018-08-31 04:39:16.848  reference  0.968885       0.001811  0.970205  ...   
8  2018-09-30 11:15:16.848  reference  0.969227       0.001811  0.974096  ...   
9  2018-10-30 17:51:16.848  reference  0.969002       0.001811  0.971757  ...   
10 2018-11-30 00:27:16.848   analysis  0.968631       0.001811       NaN  ...   
11 2018-12-30 07:03:16.848   analysis  0.969044       0.001811       NaN  ...   
12 2019-01-29 13:39:16.848   analysis  0.969444       0.001811       NaN  ...   
13 2019-02-28 20:15:16.848   analysis  0.969047       0.001811       NaN  ...   
14 2019-03-31 02:51:16.848   analysis  0.968873       0.001811       NaN  ...   
15 2019-04-30 09:27:16.848   analysis  0.960478       0.001811       NaN  ...   
16 2019-05-30 16:03:16.848   analysis  0.961134       0.001811       NaN  ...   
17 2019-06-29 22:39:16.848   analysis  0.960536       0.001811       NaN  ...   
18 2019-07-30 05:15:16.848   analysis  0.961869       0.001811       NaN  ...   
19 2019-08-29 11:51:16.848   analysis  0.960537       0.001811       NaN  ...   

    false_positive        false_negative                           \
   lower_threshold  alert          value sampling_error  realized   
0         0.032946  False       0.067159       0.004989  0.066743   
1         0.032946  False       0.070295       0.004989  0.072741   
2         0.032946  False       0.070216       0.004989  0.066771   
3         0.032946  False       0.068174       0.004989  0.070019   
4         0.032946  False       0.069470       0.004989  0.077948   
5         0.032946  False       0.067408       0.004989  0.063920   
6         0.032946  False 

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+-------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+-------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+-------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+--------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+--------------------+--------------------+--------------+-------------------------------+-------------------------------+---------------------+--------------------

In [ ]:
metric_fig = results.plot()
metric_fig.show()

In [ ]:
metric_fig = results.plot()
metric_fig.write_image(file="../_static/tutorials/performance_estimation/binary-car-loan.svg")